In [1]:
# отключим всякие предупреждения Anaconda
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from sklearn.metrics.regression import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LinearRegression, LassoCV, Lasso
from sklearn.ensemble import RandomForestRegressor

In [3]:
data = pd.read_csv('winequality-white.csv')
data.head()


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
fixed acidity           4898 non-null float64
volatile acidity        4898 non-null float64
citric acid             4898 non-null float64
residual sugar          4898 non-null float64
chlorides               4898 non-null float64
free sulfur dioxide     4898 non-null float64
total sulfur dioxide    4898 non-null float64
density                 4898 non-null float64
pH                      4898 non-null float64
sulphates               4898 non-null float64
alcohol                 4898 non-null float64
quality                 4898 non-null int64
dtypes: float64(11), int64(1)
memory usage: 459.3 KB


Отделите целевой признак, разделите обучающую выборку в отношении 7:3 (30% - под оставленную выборку, пусть random_state=17) и отмасштабируйте данные с помощью StandardScaler.


In [6]:
y=data["quality"]
data.drop("quality",axis=1,inplace=True)
data.head(1).T

,0
fixed acidity,7.000
volatile acidity,0.270
citric acid,0.360
residual sugar,20.700
chlorides,0.045
free sulfur dioxide,45.000
total sulfur dioxide,170.000
density,1.001
pH,3.000
sulphates,0.450


X_train, X_holdout, y_train, y_holdout=train_test_split(data, y, test_size=0.3, random_state=17)
scaler=StandardScaler()
scaler.fit(X_train)

In [10]:
X_train, X_holdout, y_train, y_holdout=train_test_split(data, y, test_size=0.3, random_state=17)
scaler=StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_holdout_scaled = scaler.transform(X_holdout)

Обучите простую линейную регрессию.



In [11]:
linreg = LinearRegression()
linreg.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

#### Вопрос 1: Каковы среднеквадратичные ошибки линейной регрессии на обучающей и отложенной выборках?



In [13]:
lr_train_pred=linreg.predict(X_train)
lr_test_pred=linreg.predict(X_holdout)

In [14]:
print("Mean squared error (train): %.3f" % mean_squared_error(lr_train_pred,y_train))
print("Mean squared error (test): %.3f" % mean_squared_error(lr_test_pred,y_test))

Mean squared error (train): 0.558
Mean squared error (test): 1.023


Посмотрите на коэффициенты модели и отранжируйте признаки по влиянию на качество вина (учтите, что большие по модулю отрицательные значения коэффициентов тоже говорят о сильном влиянии). Создайте для этого новый небольшой DataFrame.

#### Вопрос 2: Какой признак линейная регрессия считает наиболее сильно влияющим на качество вина?


In [51]:
linreg_coef = pd.DataFrame (data=linreg.coef_.reshape(1,-1),index=[0],columns=data.columns.values)
linreg_coef.iloc[0,:].sort_values()

density                -227.551202
volatile acidity         -1.886874
citric acid              -0.001485
total sulfur dioxide      0.000333
free sulfur dioxide       0.002439
alcohol                   0.105264
residual sugar            0.108183
fixed acidity             0.113731
chlorides                 0.352051
sulphates                 0.552495
pH                        0.995830
Name: 0, dtype: float64

## Lasso-регрессия

Обучите Lasso-регрессию с небольшим коэффициентом α=0.01 (слабая регуляризация). Пусть опять random_state=17.

In [52]:
lasso1 = Lasso (alpha=0.01,random_state=17)
lasso1.fit(X_train,y_train)

Lasso(alpha=0.01, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=17,
   selection='cyclic', tol=0.0001, warm_start=False)

Посмотрите на коэффициенты модели и отранжируйте признаки по влиянию на качество вина. Какой признак "отвалился" первым, то есть наименее важен для объяснения целевого признака в модели Lasso?



In [56]:
ind=np.argsort(abs(lasso1.coef_))
print(lasso1.coef_[ind])
data.columns.values[ind]

[ 0.         -0.         -0.          0.          0.         -0.00110804
  0.00522263  0.02223106 -0.05977011  0.35208333 -0.96790921]


array(['citric acid', 'chlorides', 'density', 'pH', 'sulphates',
       'total sulfur dioxide', 'free sulfur dioxide', 'residual sugar',
       'fixed acidity', 'alcohol', 'volatile acidity'], dtype=object)

Теперь определите лучшее значение α в процессе кросс-валидации 5-кратной кросс-валидации. Используйте LassoCV и random_state=17.

In [106]:
alphas = np.logspace(-6, 2, 200)
lasso_cv = LassoCV(alphas=alphas,cv=5)
lasso_cv.fit(X_train,y_train)

LassoCV(alphas=array([1.00000e-06, 1.09699e-06, ..., 9.11589e+01, 1.00000e+02]),
    copy_X=True, cv=5, eps=0.001, fit_intercept=True, max_iter=1000,
    n_alphas=100, n_jobs=None, normalize=False, positive=False,
    precompute='auto', random_state=None, selection='cyclic', tol=0.0001,
    verbose=False)

In [107]:
lasso_cv.alpha_

1e-06

Выведите коэффициенты "лучшего" Lasso в порядке убывания влияния на качество вина. 


In [108]:
mean_mse=np.apply_along_axis(np.mean,1,lasso_cv.mse_path_)
ind=np.argsort(mean_mse)
lasso_cv.alphas_[ind]


array([1.00000000e-06, 1.09698580e-06, 1.20337784e-06, 1.32008840e-06,
       1.44811823e-06, 1.58856513e-06, 1.74263339e-06, 1.91164408e-06,
       2.09704640e-06, 2.30043012e-06, 2.52353917e-06, 2.76828663e-06,
       3.03677112e-06, 3.33129479e-06, 3.65438307e-06, 4.00880633e-06,
       4.39760361e-06, 4.82410870e-06, 5.29197874e-06, 5.80522552e-06,
       6.36824994e-06, 6.98587975e-06, 7.66341087e-06, 8.40665289e-06,
       9.22197882e-06, 1.01163798e-05, 1.10975250e-05, 1.21738273e-05,
       1.33545156e-05, 1.46497140e-05, 1.60705282e-05, 1.76291412e-05,
       1.93389175e-05, 2.12145178e-05, 2.32720248e-05, 2.55290807e-05,
       2.80050389e-05, 3.07211300e-05, 3.37006433e-05, 3.69691271e-05,
       4.05546074e-05, 4.44878283e-05, 4.88025158e-05, 5.35356668e-05,
       5.87278661e-05, 6.44236351e-05, 7.06718127e-05, 7.75259749e-05,
       8.50448934e-05, 9.32930403e-05, 1.02341140e-04, 1.12266777e-04,
       1.23155060e-04, 1.35099352e-04, 1.48202071e-04, 1.62575567e-04,
      

#### Вопрос 3: Какой признак "обнулился первым" в настроенной модели LASSO?

In [109]:
ind=np.argsort(abs(lasso_cv.coef_))
print(lasso_cv.coef_[ind])
data.columns.values[ind]

[ 3.14166676e-04 -2.44425025e-03  2.46628936e-03  1.06579118e-01
  1.10345260e-01  1.10440187e-01  3.31071245e-01  5.46349237e-01
  9.81016020e-01 -1.88845064e+00 -2.23055887e+02]


array(['total sulfur dioxide', 'citric acid', 'free sulfur dioxide',
       'residual sugar', 'fixed acidity', 'alcohol', 'chlorides',
       'sulphates', 'pH', 'volatile acidity', 'density'], dtype=object)

#### Вопрос 4: Каковы среднеквадратичные ошибки настроенной LASSO-регрессии на обучающей и отложенной выборках?



In [111]:
lcv_train_pred=lasso_cv.predict(X_train)
lcv_test_pred=lasso_cv.predict(X_holdout)
print("Mean squared error (train): %.3f" % mean_squared_error(lcv_train_pred,y_train))
print("Mean squared error (test): %.3f" % mean_squared_error(lcv_test_pred,y_test))

Mean squared error (train): 0.558
Mean squared error (test): 1.022


## Случайный лес

Обучите случайный лес с параметрами "из коробки", фиксируя только random_state=17.



In [113]:
forest = RandomForestRegressor(random_state=17)
forest.fit(X_train,y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=17, verbose=0, warm_start=False)

#### Вопрос 5: Каковы среднеквадратичные ошибки случайного леса на обучающей выборке, на кросс-валидации (F с scoring='neg_mean_squared_error' и остальными параметрами по умолчанию) и на отложенной выборке?



In [115]:
cross_val_score(forest,X_train,y_train,scoring='neg_mean_squared_error')

array([-0.51040245, -0.43199475, -0.44222417])

In [120]:
rf_score=np.mean(cross_val_score(forest,X_train,y_train,scoring='neg_mean_squared_error'))
rf_train_pred=forest.predict(X_train)
rf_test_pred=forest.predict(X_holdout)
print("Mean squared error (train): %.3f" % mean_squared_error(rf_train_pred,y_train))
print("Mean squared error (cv): %.3f" % rf_score)
print("Mean squared error (test): %.3f" % mean_squared_error(rf_test_pred,y_test))

Mean squared error (train): 0.076
Mean squared error (cv): -0.462
Mean squared error (test): 1.185


Настройте параметры min_samples_leaf и max_depth с помощью GridSearchCV и опять проверьте качество модели на кросс-валидации и на отложенной выборке.



In [123]:
#forest_params = {'max_depth': list(range(10, 25)), 
#                 'min_samples_leaf': list(range(1, 8)),
#                 'max_features': list(range(6,12))}
forest_params={'max_features': list(range(6,12))}
locally_best_forest = GridSearchCV (forest,forest_params)
locally_best_forest.fit(X_train,y_train)

GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=17, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'max_features': [6, 7, 8, 9, 10, 11]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [126]:
locally_best_forest.best_params_, locally_best_forest.best_score_


({'max_features': 7}, 0.4203774705355304)

#### Вопрос 6: Каковы среднеквадратичные ошибки настроенного случайного леса на обучающей выборке, на кросс-валидации (cross_val_score с scoring='neg_mean_squared_error') и на отложенной выборке?

In [128]:
lrf_score=np.mean(cross_val_score(locally_best_forest,X_train,y_train,scoring='neg_mean_squared_error'))
lrf_train_pred=locally_best_forest.predict(X_train)
lrf_test_pred=locally_best_forest.predict(X_holdout)
print("Mean squared error (train): %.3f" % mean_squared_error(lrf_train_pred,y_train))
print("Mean squared error (cv): %.3f" % lrf_score)
print("Mean squared error (test): %.3f" % mean_squared_error(lrf_test_pred,y_test))

Mean squared error (train): 0.077
Mean squared error (cv): -0.460
Mean squared error (test): 1.162


Оцените важность признаков с помощью случайного леса.
#### Вопрос 7: Какой признак оказался главным в настроенной модели случайного леса?



In [136]:
imp=locally_best_forest.best_estimator_.feature_importances_
ind=np.flip(np.argsort(imp))
print(imp[ind])
print(data.columns.values[ind])

[0.22733369 0.11777514 0.11179527 0.07321616 0.07316469 0.07171586
 0.0712499  0.07094106 0.06337424 0.06223566 0.05719833]
['alcohol' 'volatile acidity' 'free sulfur dioxide' 'total sulfur dioxide'
 'density' 'pH' 'chlorides' 'residual sugar' 'citric acid' 'fixed acidity'
 'sulphates']
